<a href="https://colab.research.google.com/github/zelaza/DALL-E/blob/master/ESRGAN_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESRGAN (new arch.) on Colab
Official Github Repo: https://github.com/xinntao/ESRGAN <br>
This notebook was curated by M. Ahabb (Ahabbscience Studio)

In [1]:
!nvidia-smi #recommended gpus are p100 and T4

Fri May 27 18:03:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Clone ESRGAN repo

In [2]:
!git clone https://github.com/xinntao/ESRGAN


Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 225 (delta 4), reused 2 (delta 2), pack-reused 216
Receiving objects: 100% (225/225), 24.86 MiB | 36.89 MiB/s, done.
Resolving deltas: 100% (84/84), done.


This is optional if you want to link your google drive to the notebook to add files or pretrained models of your choice from your google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

(Optional) Add Files via youtube_dl

In [ ]:
!pip install youtube_dl

In [ ]:
!mkdir -p /content/ESRGAN/video_input
!youtube-dl -f 18 -o "/content/ESRGAN/video_input/video.mp4" https://www.youtube.com/watch?v=ljwTaMfORzs 

Run this cell to upload images

In [3]:
%cd /content/ESRGAN/LR

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

/content/ESRGAN/LR


Saving arttest.png to arttest.png
User uploaded file "arttest.png" with 454527 bytes


Run this cell if you want to upload your video. Make sure your video filename contains no spaces, e.g, "my_video" - not "my video". And upload one video per operation.

In [ ]:
!mkdir -p /content/ESRGAN/video_input
%cd /content/ESRGAN/video_input

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

/content/ESRGAN/video_input


Saving video.avi to video.avi
User uploaded file "video.avi" with 73618 bytes


Run this cell to convert your video to individual frames and save them to 'LR' folder.

In [ ]:
%cd /content/ESRGAN/LR
!ffmpeg  -i /content/ESRGAN/video_input/* %04d.png

/content/ESRGAN/LR
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --e

Download pretrained models

In [4]:
import gdown
print("Downloading pretrained models")
output1 = '/content/ESRGAN/models/RRDB_ESRGAN_x4.pth'
output2 = '/content/ESRGAN/models/RRDB_PSNR_x4.pth'
output3 = '/content/ESRGAN/models/PPON_D.pth'
output4 = '/content/ESRGAN/models/PPON_G.pth'
print ('Downloading RRDB_ESRGAN_x4.pth')
gdown.download('https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene', output1, quiet=True) 
print ('Downloading RRDB_PSNR_x4.pth')
gdown.download('https://drive.google.com/uc?id=1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN', output2, quiet=True)
print ('Downloading PPON_D.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=1Fr5aKCD6mw6P-hI0BZr6My2gHNhtUk-V', output3, quiet=True) 
print ('Downloading PPON_G.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=12uR3BSftNA0HDYiKda23GyAj_crpSjOm', output4, quiet=True)  

'/content/ESRGAN/models/PPON_G.pth'

Configure the code and run the cell to upscale images/video frames

In [10]:
#@title Upscale images/video frames
%cd /content/ESRGAN

import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests
import imageio
import requests
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

Choose_device = "cuda"  #@param ["cuda","cpu"]

model_path = 'models/RRDB_ESRGAN_x4.pth' #@param ['models/RRDB_ESRGAN_x4.pth','models/RRDB_PSNR_x4.pth','models/PPON_G.pth','models/PPON_D.pth']  
device = torch.device(Choose_device) 


test_img_folder = 'LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}.png'.format(base), output)

/content/ESRGAN
Model path models/RRDB_ESRGAN_x4.pth. 
Testing...
1 baboon
2 comic
3 arttest


Run this cell to encode your results into a video.

In [ ]:
!ffmpeg -f image2 -framerate 25 -i /content/ESRGAN/results/%04d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p output.mp4

Make a zip file of your results

In [11]:
from google.colab import files

!zip -r results.zip /content/ESRGAN/results

updating: content/ESRGAN/results/ (stored 0%)
updating: content/ESRGAN/results/baboon.png (deflated 0%)
updating: content/ESRGAN/results/comic.png (deflated 0%)
updating: content/ESRGAN/results/arttest.png (deflated 0%)
updating: content/ESRGAN/results/baboon_ESRGAN.png (deflated 0%)


Download Zip file

In [12]:
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import shutil

#move zip file to google drive
shutil.move("./results.zip", "/content/gdrive/My Drive/results.zip")